In [1]:
from officelib.xllib import *

In [35]:
def _stripify(c):
    if c is None: return ""
    t = c.strip().split(" ")
    return " ".join(t[2:])

def _unpack(row):
    return [c or "" for c in row]

def _parse_row(row):
    feature, fd, req, test = _unpack(row)
    feature = _stripify(feature)
    test = _stripify(test)
    return feature, test

def _superfit(c):
    c.ColumnWidth = 255
    c.AutoFit()

def _worksheet2(wb):
    if wb.Worksheets.Count < 2:
        return wb.Worksheets.Add()
    return wb.Worksheets(2)
    
def xfer(xl, wb, ws, cells, cr):
    # get data for copy to sheet2
    tl = cr("A2")
    br = tl.End(xlc.xlDown).GetOffset(0, 3)

    # (Feature, Functional Design, System Requirements, Test Cases)
    data = cr(tl, br).Value2

    from collections import defaultdict

    # paste data
    ws2 = _worksheet2(wb)
    cr2 = ws2.Cells.Range
    ws2.UsedRange.Clear()

    fmap = defaultdict(set)
    for row in data:
        feature, test = _parse_row(row)
        fmap[feature].add(test)
    
    paste_data = []
    for feature, tests in fmap.items():
        tests_list = "\n".join(sorted(filter(None, tests)))
        if not tests_list:
            pass
#             continue
        row = (feature, tests_list)
        paste_data.append(row)
    
    tl2 = cr2("A2")
    br2 = tl2.GetOffset(len(paste_data) - 1, len(paste_data[0]) - 1)
    
    cr2(tl2, br2).Value2 = paste_data
    
    cr2("A1:B1").Value2 = [("Feature", "Factory & OQ Test Sections")]
    
    for col in (1,2):
        _superfit(ws2.Columns(col))
    
    ur = ws2.UsedRange
    ur.Rows.AutoFit()
    ur.VerticalAlignment = xlc.xlTop
    

In [38]:
def import_helix_mtrx(xl):
    wb = xl.ActiveWorkbook
    ws = wb.Worksheets(1)
    cells = ws.Cells
    cr = cells.Range

    with screen_lock(xl):
        xfer(xl, wb, ws, cells, cr)
    
xl = Excel()
import_helix_mtrx(xl)